In [1]:
import sys
from pathlib import Path
import os
import json

sys.path.append(str(Path.cwd().parent.parent))
print(str(Path.cwd().parent.parent))
print("Current working directory: ", os.getcwd())

from DataProcessing.DataAugmentation import ImageEnhancement
from ModelDevelopment.CentralPipeline import CentralPipeline
from ModelDevelopment.ImageBatchPipeline import ImageBatchPipeline
from DataProcessing.DataPreProcessing import DataPaths

c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition
Current working directory:  c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\ModelDevelopment\experiments


In [2]:
pipeline = CentralPipeline(
  #num_tracklets=1211, #1426,
  #num_images_per_tracklet=50,
  #tracklets_to_process_override=tracklets_to_process_override_batch1,
  #tracklets_to_process_override=[109, 88, 34, 151, 19, 68, 67, 128, 57, 178, 189, 196, 214, 242, 258, 271, 304, 327, 320, 352, 375, 378, 414, 416, 439, 425, 451, 453, 454, 466, 499, 511, 513, 526, 528, 567, 576, 594, 619, 623, 646, 687, 682, 709, 711, 717, 739, 758, 767, 795, 788, 840, 843, 832, 841, 852, 847, 851, 856, 850, 868, 866, 871, 901, 902, 904, 900, 907, 928, 939, 944, 961, 984, 983, 1005, 1039, 1064, 1063, 1077, 1091, 1101, 1137, 1152, 1153, 1160, 1167, 1157, 1165, 1206],
  tracklets_to_process_override=['11', '12', '18', '19', '26', '34', '46', '52', '54', '57', '67', '68', '88', '102', '109', '128', '136', '138', '145', '148', '150', '151', '166', '178', '189', '196', '214', '219', '232', '242', '253', '258', '271', '304', '318', '320', '327', '349', '352', '361', '375', '378', '385', '397', '413', '414', '416', '425', '439', '451', '452', '453', '454', '461', '464', '466', '489', '494', '496', '498', '499', '506', '510', '511', '512', '513', '526', '528', '546', '567', '576', '594', '608', '619', '623', '646', '661', '675', '682', '687', '701', '702', '709', '711', '717', '725', '735', '739', '758', '767', '777', '788', '795', '832', '840', '841', '843', '847', '850', '851', '852', '855', '856', '863', '866', '868', '871', '882', '883', '886', '890', '891', '900', '901', '902', '904', '906', '907', '911', '914', '915', '928', '939', '940', '944', '957', '961', '963', '978', '980', '983', '984', '986', '1005', '1009', '1013', '1015', '1029', '1039', '1049', '1050', '1051', '1054', '1061', '1062', '1063', '1064', '1074', '1077', '1091', '1101', '1104', '1109', '1110', '1114', '1128', '1137', '1148', '1152', '1153', '1154', '1157', '1160', '1165', '1167', '1181'],
  input_data_path=DataPaths.TEST_DATA_DIR.value, #DataPaths.CHALLENGE_DATA_DIR.value,
  output_processed_data_path=DataPaths.PROCESSED_DATA_OUTPUT_DIR_TEST.value, #DataPaths.PROCESSED_DATA_OUTPUT_DIR_CHALLENGE.value,
  common_processed_data_dir=DataPaths.COMMON_PROCESSED_OUTPUT_DATA_TEST.value, #DataPaths.COMMON_PROCESSED_OUTPUT_DATA_CHALLENGE.value,
  gt_data_path=DataPaths.TEST_DATA_GT.value, #"",
  use_cache=True, # Set to false to rebuild the cache
  suppress_logging=False,
  use_image_enhancement=True,
  # --- PARALLELIZATION PARAMS --- These settings are optimal for an NVIDIA RTX 3070 Ti Laptop GPU.
  num_workers=1,                   # CRITICAL optimisation param. Adjust accordingly. STR: 1 | Everything else: 6
  tracklet_batch_size=32,          # CRITICAL optimisation param. Adjust accordingly. Always 32
  image_batch_size=1000,           # CRITICAL optimisation param. Adjust accordingly.
  num_threads_multiplier=2         # CRITICAL optimisation param. Adjust accordingly. Always 6
  )                                # -> Interpretation: num_threads = num_workers * num_threads_multiplier

2025-04-04 22:15:25 [INFO] DataPreProcessing initialized. Universe of available data paths:
2025-04-04 22:15:25 [INFO] ROOT_DATA_DIR: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted
2025-04-04 22:15:25 [INFO] TEST_DATA_GT: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\test\test_gt.json
2025-04-04 22:15:25 [INFO] TRAIN_DATA_GT: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\train\train_gt.json
2025-04-04 22:15:25 [INFO] TEST_DATA_DIR: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\test\images
2025-04-04 22:15:25 [INFO] TRAIN_DATA_DIR: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\train\images
2025-04-04 22:15:25 [INFO] CHALLENGE_DATA_DIR: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\challenge\images
2

In [3]:
#pipeline.aggregate_legibility_results_data()
#pipeline.aggregate_pose()
#pipeline.aggregate_str_results()

In [4]:
pipeline.run_soccernet(
  run_soccer_ball_filter=False,
  generate_features=False,
  run_filter=False,
  run_legible=False,
  run_legible_eval=False,
  run_pose=False,
  run_crops=False,
  run_str=False,
  run_combine=True,
  run_eval=True,
  use_clip4str=True,
  pyscript=False,
  )
#pipeline.aggregate_str_results()

2025-04-04 22:15:26 [INFO] Running the SoccerNet pipeline.
2025-04-04 22:15:26 [INFO] Reading legible & illegible results from cache (both global files exist).
2025-04-04 22:15:26 [INFO] Legible tracklets list: 11, 12, 18, 26, 46, 52, 54, 102, 136, 138, 145, 148, 150, 166, 219, 232, 253, 318, 327, 349, 361, 385, 397, 413, 452, 461, 464, 489, 494, 496, 498, 506, 510, 512, 546, 608, 661, 675, 701, 702, 711, 725, 735, 777, 855, 863, 882, 883, 886, 890, 891, 906, 907, 911, 914, 915, 940, 957, 963, 978, 980, 986, 1009, 1013, 1015, 1029, 1049, 1050, 1051, 1054, 1061, 1062, 1064, 1074, 1104, 1109, 1110, 1114, 1128, 1148, 1154, 1181
2025-04-04 22:15:26 [INFO] Results dict: {'18': '8', '26': '9', '12': '33', '46': '36', '11': -1, '52': '36', '54': '3', '136': '3', '102': '59', '138': '1', '145': '-1', '148': '-1', '150': '-1', '166': '34', '219': '1', '232': '10', '253': '27', '318': '-1', '327': '-1', '349': '34', '361': '59', '397': '36', '385': -1, '413': '-1', '452': -1, '461': '2', '464': 

In [5]:
# import json

# with open("C:/Users/colin/OneDrive/Desktop/Jersey-Number-Recognition/data/SoccerNet/jersey-2023/processed_data/challenge/common_data/final_results.json", "r") as f:
#     data = json.load(f)
    
# # Create the expected keys range (as strings to match JSON format)
# missing_keys = []
# for i in range(1426):
#   if str(i) not  in data:
#     missing_keys.append(i)
    
# if missing_keys:
#   print("Missing keys: ", missing_keys)
# else:
#   print("All keys are present.")

## Benchmark (current pipeline metrics)
### Results
- 2025-03-29 14:44:05 [INFO] Correct 1051 out of 1210. Accuracy 86.85950413223141%.
- 2025-03-29 14:44:05 [INFO] TP=766, TN=285, FP=70, FN=89
- 2025-03-29 14:44:05 [INFO] Precision=0.916267942583732, Recall=0.895906432748538
- 2025-03-29 14:44:05 [INFO] F1=0.9059727971614429
- 2025-03-29 14:44:05 [INFO] Saved failed legibility cases to:  c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\processed_data\test\common_data\failed_legibility_cases.json

### Analysis
legible_but_marked_illegible: 89 cases
109, 88, 34, 151, 19, 68, 67, 128, 57, 178, 189, 196, 214, 242, 258, 271, 304, 327, 320, 352, 375, 378, 414, 416, 439, 425, 451, 453, 454, 466, 499, 511, 513, 526, 528, 567, 576, 594, 619, 623, 646, 687, 682, 709, 711, 717, 739, 758, 767, 795, 788, 840, 843, 832, 841, 852, 847, 851, 856, 850, 868, 866, 871, 901, 902, 904, 900, 907, 928, 939, 944, 961, 984, 983, 1005, 1039, 1064, 1063, 1077, 1091, 1101, 1137, 1152, 1153, 1160, 1167, 1157, 1165, 1206

illegible_but_marked_legible: 70 cases
168, 167, 162, 97, 39, 87, 89, 134, 61, 172, 164, 199, 215, 260, 267, 269, 281, 276, 288, 295, 317, 325, 340, 345, 374, 394, 388, 386, 391, 406, 408, 434, 437, 459, 471, 514, 532, 558, 683, 748, 770, 790, 802, 827, 861, 869, 879, 897, 895, 909, 964, 981, 999, 1023, 1031, 1025, 1033, 1030, 1055, 1088, 1087, 1169, 1175, 1174, 1173, 1183, 1170, 1179, 1184, 1185

- Total misclassifications: 159
- False negatives (legible marked as illegible): 89
- False positives (illegible marked as legible): 70
- Percentage of false negatives: 7.36%
- Percentage of false positives: 5.79%
- Combined percentage of errors as a percentage of total cases: 13.14%

In [6]:
# import json
# failed_cases = f"{DataPaths.COMMON_PROCESSED_OUTPUT_DATA_CHALLENGE.value}/failed_legibility_cases.json"

# # Load this JSON file
# with open(failed_cases, 'r') as f:
#     failed_cases_dict = json.load(f)
#     # Print the every key and its associated value (value will be a list so need to make that printable with ",".join):
#     for key, value in failed_cases_dict.items():
#         print(f"{key}: {len(value)} cases")
#         if len(value) > 0:
#             print(f"{', '.join(value)}")
#         print()
    
#     # Summary statistics
#     legible_but_marked_illegible = len(failed_cases_dict.get("legible_but_marked_illegible", []))
#     illegible_but_marked_legible = len(failed_cases_dict.get("illegible_but_marked_legible", []))
#     total_errors = legible_but_marked_illegible + illegible_but_marked_legible
    
#     print(f"Total misclassifications: {total_errors}")
#     print(f"False negatives (legible marked as illegible): {legible_but_marked_illegible}")
#     print(f"False positives (illegible marked as legible): {illegible_but_marked_legible}")
#     print(f"Percentage of false negatives: {legible_but_marked_illegible / 1210 * 100:.2f}%")
#     print(f"Percentage of false positives: {illegible_but_marked_legible / 1210 * 100:.2f}%")
#     print(f"Combined percentage of errors as a percentage of total cases: {total_errors / 1210 * 100:.2f}%")

In [7]:
# import timm
# import torch
# from torch import nn

# model = timm.create_model("timm/vit_base_patch16_224.orig_in21k_ft_in1k", pretrained=False)
# model.head = nn.Linear(model.head.in_features, 10)
# model.load_state_dict(
#     torch.hub.load_state_dict_from_url(
#         "https://huggingface.co/edadaltocg/vit_base_patch16_224_in21k_ft_svhn/resolve/main/pytorch_model.bin",
#         map_location="cpu",
#         file_name="vit_base_patch16_224_in21k_ft_svhn.pth",
#     )
# )

In [8]:
pct_83_test = r"C:\Users\colin\OneDrive\Desktop\test_run_83pct\final_results.json"
gt = DataPaths.TEST_DATA_GT.value
with open(pct_83_test, "r") as f:
    data = json.load(f)
    
gt_dict = {}
with open(gt, "r") as f:
    gt_data = json.load(f)
        
# Now retrieve the keys for which the values are not -1 from the ground truth, and the value does not align with our predicted one
wrong_predictions = []
for key, value in gt_data.items():
    # Check if they ley exists first before accessing
    if key not in data:
        print(f"Key {key} not found in predictions")
        continue
    
    if value != -1 and value != data[key]:
        wrong_predictions.append(key)
        
# Print the keys that are wrong
print("Wrong predictions: ", wrong_predictions)

Key 1210 not found in predictions
Wrong predictions:  ['11', '12', '18', '19', '26', '34', '46', '52', '54', '57', '67', '68', '88', '102', '109', '128', '136', '138', '145', '148', '150', '151', '166', '178', '189', '196', '214', '219', '232', '242', '253', '258', '271', '304', '318', '320', '327', '349', '352', '361', '375', '378', '385', '397', '413', '414', '416', '425', '439', '451', '452', '453', '454', '461', '464', '466', '489', '494', '496', '498', '499', '506', '510', '511', '512', '513', '526', '528', '546', '567', '576', '594', '608', '619', '623', '646', '661', '675', '682', '687', '701', '702', '709', '711', '717', '725', '735', '739', '758', '767', '777', '788', '795', '832', '840', '841', '843', '847', '850', '851', '852', '855', '856', '863', '866', '868', '871', '882', '883', '886', '890', '891', '900', '901', '902', '904', '906', '907', '911', '914', '915', '928', '939', '940', '944', '957', '961', '963', '978', '980', '983', '984', '986', '1005', '1009', '1013', '10